In [1]:
import pandas as pd
from funciones import *
import datetime

In [2]:
rutaI=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\P&G Industria\05. P&G\Venta/"
rutaCV=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes Base\{}\Costo de Ventas/{}. Costo de Ventas Industria CEBE.xlsx"
rutaM=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\P&G Industria\05. P&G\Maestras/"

rutaD=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes\P&G/"

In [3]:
cols=['Sublinea ID', 'Sublinea DESC', 'Plu PluCD', 'Plu DESC', '$ Costo','# Unidades Totales', '$ Ventas sin impuestos']

conv={'Sublinea ID':str,"Plu PluCD":str}

In [4]:
mesA=12
yearA=2022
mesI=5
yearI=2023

In [5]:
for tiempo in month_year_iter(mesA,yearA,mesI,yearI):
    df=pd.read_excel(rutaI+"{}{:02d}. Ingresos.xlsx".format(tiempo[0],tiempo[1]),usecols=cols,converters=conv)
    df=df.rename(columns={"Plu PluCD":"PLU",'$ Costo':"Costo (Sinemax)",
                          '# Unidades Totales':"Cantidad",
                          '$ Ventas sin impuestos':"Ingreso",
                         "Sublinea DESC":'Descripción Sublinea',
                         'Plu DESC':'Descripción PLU'})

    df=df.groupby(['Sublinea ID', 'Descripción Sublinea', 'PLU', 'Descripción PLU']).sum().reset_index()
    df["Fecha"]=datetime.date(tiempo[0],tiempo[1],1)
    
    dfAux=pd.read_excel(rutaM+"PLU SAP SUBLINEA.xlsx",usecols=["Material","PLU"],
                       converters={"Material":str,"PLU":str})

    if df.merge(dfAux,on=["PLU"],how="left").shape[0]!=df.shape[0]:
        raise Exception("Agrega Filas")

    df=df.merge(dfAux,on=["PLU"],how="left")
    df=df[~df["Material"].isna()]
    
    dfCV=pd.read_excel(rutaCV.format(tiempo[0],tiempo[1]),usecols=["Número de cuenta","Centro de beneficio","Material",
                                                      "En moneda local centro de beneficio","Cantidad"],
                       converters={"Número de cuenta":str,"Centro de beneficio":str,"Material":str})
    dfCV["Material"].fillna("Otros",inplace=True)

    dfCV=dfCV.groupby(["Número de cuenta","Centro de beneficio","Material"]).sum().reset_index()

    dfAux=pd.read_excel(rutaM+"Grupo Cuentas.xlsx",usecols=["Número de cuenta","Línea P&G"],
                       converters={"Número de cuenta":str})

    dfCV=dfCV.merge(dfAux,on=["Número de cuenta"],how="left")

    dfCV=dfCV[~dfCV["Línea P&G"].isna()]

    del dfCV["Número de cuenta"]

    if dfCV.isna().sum().sum()>0:
        raise Exception("Na encontrado")

    dfCV=dfCV.groupby(["Centro de beneficio","Material",'Línea P&G']).sum().reset_index()

    dfAux=pd.read_excel(rutaM+"CebeSub.xlsx",usecols=["Centro de beneficio","Sublinea ID","Ocurrencias"],
                       converters={"Centro de beneficio":str,"Sublinea ID":str})

    dfCV=dfCV.merge(dfAux,on=["Centro de beneficio"],how="left")

    dfCV["En moneda local centro de beneficio"]=dfCV["En moneda local centro de beneficio"].divide(dfCV["Ocurrencias"])
    dfCV["Cantidad"]=dfCV["Cantidad"].divide(dfCV["Ocurrencias"])

    del dfCV["Ocurrencias"]
    del dfCV["Centro de beneficio"]

    if dfCV.isna().sum().sum()>0:
        raise Exception("Na encontrado1")
    dfCV=dfCV.groupby(["Sublinea ID","Material",'Línea P&G']).sum().reset_index()
    ##Step1
    ## Costo Producto

    temp=dfCV.copy()

    temp=temp[temp["Línea P&G"]=='Costo Producto']

    del temp["Sublinea ID"]
    del temp["Línea P&G"]
    temp=temp.groupby(["Material"]).sum().reset_index()

    temp["Costo Unitario"]=temp["En moneda local centro de beneficio"].divide(temp["Cantidad"])

    if df.merge(temp[["Material","Costo Unitario"]],on=["Material"],how="left").shape[0] != df.shape[0]:
        raise Exception("Ingresa Filas 1")

    df=df.merge(temp[["Material","Costo Unitario"]],on=["Material"],how="left")
    df=df[~df["Costo Unitario"].isna()]

    df["Costo Producto"]=df["Costo Unitario"]*df["Cantidad"]
    ##Step2
    #Otras Líneas P&G
    
    del dfCV["Cantidad"]
    dfCV=dfCV[dfCV["Línea P&G"]!='Costo Producto']

    dfCV=pd.pivot_table(dfCV,values=["En moneda local centro de beneficio"], index=["Sublinea ID","Material"],
                   columns=["Línea P&G"], aggfunc='sum',dropna=False).reset_index().fillna(0)

    cTemp=[]
    for i in dfCV.columns:
        if i[1]!="":
            cTemp.append(i[1])
        else:
            cTemp.append(i[0])


    dfCV.columns=cTemp
    
    if "Averias" not in cTemp:
        dfCV["Averias"]=0
    
    dfCV=dfCV[~((dfCV["Ajustes"]==0.0)&(dfCV["Bajas"]==0.0)&(dfCV["Mermas"]==0.0)&(dfCV["Variaciones"]==0.0)&(dfCV["Depreciación"]==0.0)&(dfCV["Averias"]==0.0))]
    dfCV=dfCV.merge(df[["Sublinea ID","Material","Ingreso"]].groupby(["Sublinea ID","Material"]).sum(),
                    how="left",on=["Sublinea ID","Material"])

    temp=dfCV[(~dfCV["Ingreso"].isna())&(dfCV["Ingreso"]!=0.0)]
    dfCV=dfCV[~((~dfCV["Ingreso"].isna())&(dfCV["Ingreso"]!=0.0))]

    for i in temp.columns:
        if i not in ["Sublinea ID","Material","Ingreso"]:
            temp[i+"_ratio"]=temp[i].divide(temp["Ingreso"])
            del temp[i]

    del temp["Ingreso"]

    if df.merge(temp,on=["Sublinea ID","Material"],how="left").shape[0]!=df.shape[0]:
        raise Exception("Inserta filas 2")
    df=df.merge(temp,on=["Sublinea ID","Material"],how="left")

    df.fillna(0,inplace=True)
    for i in df.columns:
        if "_ratio" in i:
            df[i[:-6]+"_(Dir)"]=df[i]*df["Ingreso"]
            del df[i]
    
    # material
    del dfCV["Ingreso"]
    dfCV=dfCV.merge(df[["Material","Ingreso"]].groupby(["Material"]).sum(),on="Material",how="left")

    temp=dfCV[(~dfCV["Ingreso"].isna())&(dfCV["Ingreso"]!=0.0)]
    dfCV=dfCV[~((~dfCV["Ingreso"].isna())&(dfCV["Ingreso"]!=0.0))]

    del temp["Sublinea ID"]
    del temp["Ingreso"]

    temp=temp.groupby(["Material"]).sum().reset_index()

    temp=temp.merge(df[["Material","Ingreso"]].groupby(["Material"]).sum(),on="Material",how="left")

    for i in temp.columns:
        if i not in ["Sublinea ID","Material","Ingreso"]:
            temp[i+"_ratio"]=temp[i].divide(temp["Ingreso"])
            del temp[i]

    del temp["Ingreso"]

    if df.merge(temp,on=["Material"],how="left").shape[0]!=df.shape[0]:
        raise Exception("Inserta filas 3")
    df=df.merge(temp,on=["Material"],how="left")

    df.fillna(0,inplace=True)
    for i in df.columns:
        if "_ratio" in i:
            df[i[:-6]+"_(Mat)"]=df[i]*df["Ingreso"]
            del df[i]

    del dfCV["Ingreso"]

    # sublinea
    del dfCV["Material"]

    dfCV=dfCV.groupby(["Sublinea ID"]).sum().reset_index()
    dfCV=dfCV.merge(df[['Sublinea ID','Ingreso']].groupby(["Sublinea ID"]).sum().reset_index(),on="Sublinea ID",how="left")

    temp=dfCV[(~dfCV["Ingreso"].isna())&(dfCV["Ingreso"]!=0.0)]
    dfCV=dfCV[~((~dfCV["Ingreso"].isna())&(dfCV["Ingreso"]!=0.0))]

    for i in temp.columns:
        if i not in ["Sublinea ID","Ingreso"]:
            temp[i+"_ratio"]=temp[i].divide(temp["Ingreso"])
            del temp[i]

    del temp["Ingreso"]

    if df.merge(temp,on=["Sublinea ID"],how="left").shape[0]!=df.shape[0]:
        raise Exception("Inserta filas 4")
    df=df.merge(temp,on=["Sublinea ID"],how="left")

    df.fillna(0,inplace=True)
    for i in df.columns:
        if "_ratio" in i:
            df[i[:-6]+"_(Sub)"]=df[i]*df["Ingreso"]
            del df[i]

    del dfCV["Ingreso"]

    dfCV["Ingreso"]=df["Ingreso"].sum().copy()

    for i in dfCV.columns:
        if i not in ["Sublinea ID","Ingreso"]:
            dfCV[i+"_ratio"]=dfCV[i].divide(dfCV["Ingreso"])
            del dfCV[i]
            df[i+"_(Tr)"]=dfCV[i+"_ratio"].sum()*df["Ingreso"]

    df.fillna(0,inplace=True)
    
    df.to_excel(rutaD+"{}{:02d}. P&G.xlsx".format(tiempo[0],tiempo[1]),index=None)
    print(tiempo)

(2022, 12)
(2023, 1)
(2023, 2)
(2023, 3)
(2023, 4)
